# Chapter 3.2

## Importing Libraries

In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import copy
import tqdm
from hfunc import models
from hfunc import metrics
from sklearn.model_selection import train_test_split

## Set seed

In [3]:
np.random.seed(2020)

## Self-created functions

In [4]:
def estimate_node_importance(model, tester_model, layer_sizes, tol_low, tol_high, x, y):
    
    l, a = model.evaluate(x, y, verbose=0, batch_size=256)
    or_weights = model.get_weights()
    weight_len = len(or_weights) - 3
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    amounts = []
    places = []
    for layer, size in enumerate(layer_sizes):
        num_zeros, num_worse, num_important = (0, 0, 0)
        z = []
        wr = []
        imp = []
        for i in range(size):
            w = copy.deepcopy(or_weights)
            w[weight_len - (2*layer+1)][:,i] = 0
            w[weight_len - 2*layer][i] = 0
            tester_model.set_weights(w)
            nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
            change = l - nl
            if change <= tol_high and change >= tol_low:
                num_zeros += 1
                z += [i]
            elif change > 0:
                num_worse += 1
                wr += [i]
            else:
                num_important += 1
                imp += [i]
        amounts.append((num_zeros, num_worse, num_important))
        places.append((z, wr, imp))
    
    return amounts, places

In [5]:
def estimate_node_importance_conv(model, tester_model, dense_layer_sizes, conv_layer_sizes, tol_low, tol_high, x, y):
    
    l, a = model.evaluate(x, y, verbose=0, batch_size=256)
    or_weights = model.get_weights()
    weight_len = len(or_weights) - 3
    conv_len = weight_len - 2 * len(dense_layer_sizes)
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    amounts = []
    places = []
    
    # For-loop over dense layers
    for layer, size in enumerate(dense_layer_sizes):
        num_zeros, num_worse, num_important = (0, 0, 0)
        z = []
        wr = []
        imp = []
        for i in range(size):
            w = copy.deepcopy(or_weights)
            w[weight_len - (2*layer+1)][:,i] = 0
            w[weight_len - 2*layer][i] = 0
            tester_model.set_weights(w)
            nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
            change = l - nl
            if change <= tol_high and change >= tol_low:
                num_zeros += 1
                z += [i]
            elif change > 0:
                num_worse += 1
                wr += [i]
            else:
                num_important += 1
                imp += [i]
        amounts.append((num_zeros, num_worse, num_important))
        places.append((z, wr, imp))
    
    # For-loop over convolutional layers
    for layer, size in enumerate(conv_layer_sizes):
        num_zeros, num_worse, num_important = (0, 0, 0)
        z = []
        wr = []
        imp = []
        for i in range(size):
            w = copy.deepcopy(or_weights)
            w[conv_len - (2*layer+1)][:, :, :, i] = 0
            w[conv_len - 2*layer][i] = 0
            tester_model.set_weights(w)
            nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
            change = l - nl
            if change <= tol_high and change >= tol_low:
                num_zeros += 1
                z += [i]
            elif change > 0:
                num_worse += 1
                wr += [i]
            else:
                num_important += 1
                imp += [i]
        amounts.append((num_zeros, num_worse, num_important))
        places.append((z, wr, imp))

    return amounts, places

## Single-Layer ANN

### MNIST

## Set seed

In [6]:
np.random.seed(2020)

In [27]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [8]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [36:53<00:00, 44.27s/it]


In [10]:
NI = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI.to_csv('../../../results/single_layer_node_importance_mnist.csv')
NI.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.260000,13.860000,111.880000
std,1.425855,6.003434,5.902749
min,0.000000,4.000000,100.000000
25%,1.000000,9.000000,107.250000
50%,2.500000,13.000000,112.000000
75%,3.000000,18.000000,116.750000
max,5.000000,27.000000,123.000000


### Fashion MNIST

## Set seed

In [15]:
np.random.seed(2020)

In [16]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [17]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [18]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_test, y_test), epochs=5)
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [36:44<00:00, 44.09s/it]


In [19]:
NI = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI.to_csv('../../../results/single_layer_node_importance_fmnist.csv')
NI.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,13.960000,20.920000,93.120000
std,3.362215,6.502245,7.020785
min,5.000000,8.000000,80.000000
25%,12.000000,16.000000,89.000000
50%,14.000000,19.500000,92.000000
75%,16.000000,25.750000,97.750000
max,24.000000,36.000000,115.000000


## Multi-layer Perceptron

### MNIST

## Set seed

In [20]:
np.random.seed(2020)

In [21]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [22]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [23]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [32, 64, 128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [1:00:10<00:00, 72.22s/it]


In [24]:
NI_1 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_2 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_3 = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_1.to_csv('../../../results/multi_layer_1_node_importance_mnist.csv')
NI_2.to_csv('../../../results/multi_layer_2_node_importance_mnist.csv')
NI_3.to_csv('../../../results/multi_layer_3_node_importance_mnist.csv')

In [25]:
NI_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.980000,18.480000,106.540000
std,1.857033,6.198222,6.414938
min,0.000000,2.000000,89.000000
25%,2.000000,14.250000,102.000000
50%,3.000000,19.000000,107.000000
75%,4.000000,23.000000,110.750000
max,7.000000,34.000000,125.000000


In [26]:
NI_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.060000,9.420000,52.520000
std,1.268295,4.695156,4.895812
min,0.000000,0.000000,42.000000
25%,1.000000,6.000000,49.000000
50%,2.000000,9.000000,53.000000
75%,3.000000,12.750000,56.000000
max,5.000000,20.000000,62.000000


In [27]:
NI_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.040000,2.180000,27.780000
std,1.640806,2.057242,2.565787
min,0.000000,0.000000,22.000000
25%,0.250000,1.000000,26.000000
50%,2.000000,2.000000,28.000000
75%,3.000000,3.000000,30.000000
max,5.000000,8.000000,32.000000


### Fashion MNIST

## Set seed

In [28]:
np.random.seed(2020)

In [29]:
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [30]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [31]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [32, 64, 128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|████████████████████████████████████████████████████████████████████████████████| 50/50 [1:00:39<00:00, 72.79s/it]


In [32]:
NI_1 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_2 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_3 = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_1.to_csv('../../../results/multi_layer_1_node_importance_fmnist.csv')
NI_2.to_csv('../../../results/multi_layer_2_node_importance_fmnist.csv')
NI_3.to_csv('../../../results/multi_layer_3_node_importance_fmnist.csv')

In [33]:
NI_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,17.660000,18.460000,91.880000
std,3.594838,7.019535,7.121798
min,10.000000,3.000000,75.000000
25%,15.250000,13.000000,87.000000
50%,17.500000,19.000000,92.000000
75%,19.750000,24.000000,97.000000
max,26.000000,34.000000,106.000000


In [34]:
NI_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,4.160000,12.200000,47.640000
std,2.160499,5.058837,5.255745
min,1.000000,0.000000,36.000000
25%,2.250000,9.000000,44.000000
50%,4.000000,12.000000,47.500000
75%,6.000000,15.000000,51.000000
max,10.000000,23.000000,58.000000


In [35]:
NI_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.780000,3.520000,25.700000
std,1.729575,2.366777,2.384046
min,0.000000,0.000000,20.000000
25%,2.000000,2.000000,24.000000
50%,2.000000,3.000000,26.000000
75%,4.000000,4.750000,27.750000
max,9.000000,9.000000,30.000000


## Convolutional Neural Network

### MNIST

## Set seed

In [36]:
np.random.seed(2020)

In [37]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [38]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [39]:
trials = 10
low_tol = -1e-5
high_tol = 1e-5
conv_layer_sizes = [256, 128, 64, 32]
dense_layer_sizes = [64]
node_importance = np.zeros((trials, len(dense_layer_sizes) + len(conv_layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance_conv(model, tester_model, dense_layer_sizes, conv_layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [58:39<00:00, 351.95s/it]


In [40]:
ND = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_4 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_3 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_2 = pd.DataFrame(node_importance[:, 3, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_1 = pd.DataFrame(node_importance[:, 4, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
ND.to_csv('../../../results/CNN_dense_node_importance_mnist.csv')
NC_1.to_csv('../../../results/CNN_conv_1_node_importance_mnist.csv')
NC_2.to_csv('../../../results/CNN_conv_2_node_importance_mnist.csv')
NC_3.to_csv('../../../results/CNN_conv_3_node_importance_mnist.csv')
NC_4.to_csv('../../../results/CNN_conv_4_node_importance_mnist.csv')

In [41]:
NC_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.00000,10.000000,10.000000
mean,25.70000,9.300000,29.000000
std,6.11101,5.313505,4.496913
min,19.00000,2.000000,20.000000
25%,21.50000,6.000000,27.250000
50%,24.50000,8.000000,29.500000
75%,29.00000,12.250000,32.000000
max,38.00000,18.000000,35.000000


In [42]:
NC_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,73.600000,62.300000,120.100000
std,13.574486,21.061022,19.885506
min,51.000000,27.000000,90.000000
25%,66.250000,48.000000,107.500000
50%,71.500000,62.500000,118.500000
75%,81.500000,75.750000,135.000000
max,98.000000,96.000000,148.000000


In [43]:
NC_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,14.200000,31.400000,82.400000
std,5.050853,11.403703,9.203864
min,9.000000,11.000000,66.000000
25%,10.500000,29.250000,76.750000
50%,13.000000,33.000000,83.000000
75%,15.750000,36.750000,87.500000
max,26.000000,50.000000,98.000000


In [44]:
NC_4.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,3.000000,16.600000,44.400000
std,1.632993,6.669999,6.292853
min,0.000000,8.000000,35.000000
25%,2.000000,12.250000,38.500000
50%,3.000000,16.000000,46.000000
75%,4.000000,22.000000,48.000000
max,6.000000,27.000000,54.000000


In [45]:
ND.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,4.800000,6.200000,21.000000
std,2.299758,4.491968,4.521553
min,1.000000,1.000000,13.000000
25%,4.000000,3.250000,18.000000
50%,5.000000,5.000000,21.500000
75%,5.000000,8.250000,23.750000
max,10.000000,15.000000,28.000000


### Fashion MNIST

## Set seed

In [46]:
np.random.seed(2020)

In [47]:
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]

In [48]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [49]:
trials = 10
low_tol = -1e-5
high_tol = 1e-5
conv_layer_sizes = [256, 128, 64, 32]
dense_layer_sizes = [64]
node_importance = np.zeros((trials, len(dense_layer_sizes) + len(conv_layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance_conv(model, tester_model, dense_layer_sizes, conv_layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [58:21<00:00, 350.17s/it]


In [50]:
ND = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_4 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_3 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_2 = pd.DataFrame(node_importance[:, 3, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_1 = pd.DataFrame(node_importance[:, 4, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
ND.to_csv('../../../results/CNN_dense_node_importance_fmnist.csv')
NC_1.to_csv('../../../results/CNN_conv_1_node_importance_fmnist.csv')
NC_2.to_csv('../../../results/CNN_conv_2_node_importance_fmnist.csv')
NC_3.to_csv('../../../results/CNN_conv_3_node_importance_fmnist.csv')
NC_4.to_csv('../../../results/CNN_conv_4_node_importance_fmnist.csv')

In [51]:
NC_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,25.700000,6.300000,32.000000
std,4.785394,2.869379,4.055175
min,19.000000,2.000000,26.000000
25%,22.250000,4.000000,28.750000
50%,25.500000,6.500000,32.500000
75%,29.500000,8.000000,33.750000
max,32.000000,11.000000,40.000000


In [52]:
NC_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,49.100000,48.300000,158.600000
std,3.446415,20.693799,20.429826
min,44.000000,12.000000,124.000000
25%,46.750000,32.500000,151.750000
50%,50.000000,50.500000,155.500000
75%,50.750000,60.250000,173.750000
max,54.000000,81.000000,190.000000


In [53]:
NC_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,11.300000,23.200000,93.500000
std,2.945807,7.814516,8.592633
min,6.000000,10.000000,82.000000
25%,9.500000,18.000000,86.250000
50%,11.000000,25.000000,92.000000
75%,12.750000,28.750000,99.750000
max,16.000000,34.000000,109.000000


In [54]:
NC_4.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.00000,10.000000
mean,0.900000,10.70000,52.400000
std,1.100505,6.88073,6.686637
min,0.000000,2.00000,38.000000
25%,0.000000,7.00000,50.250000
50%,0.500000,9.00000,54.500000
75%,1.750000,12.25000,55.000000
max,3.000000,26.00000,62.000000


In [55]:
ND.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,0.600000,3.900000,27.500000
std,0.843274,3.634709,3.719319
min,0.000000,0.000000,21.000000
25%,0.000000,0.500000,26.000000
50%,0.000000,3.500000,28.500000
75%,1.000000,5.750000,30.000000
max,2.000000,10.000000,32.000000


## CIFAR10

## Set seed

In [56]:
np.random.seed(2020)

In [57]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [58]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [59]:
trials = 10
low_tol = -1e-5
high_tol = 1e-5
conv_layer_sizes = [256, 128, 64, 32]
dense_layer_sizes = [64]
node_importance = np.zeros((trials, len(dense_layer_sizes) + len(conv_layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance_conv(model, tester_model, dense_layer_sizes, conv_layer_sizes, low_tol, high_tol, x_train, y_train)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [1:32:35<00:00, 555.54s/it]


In [60]:
ND = pd.DataFrame(node_importance[:, 4, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_4 = pd.DataFrame(node_importance[:, 3, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_3 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_2 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_1 = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
ND.to_csv('../../../results/CNN_dense_node_importance_cifar.csv')
NC_1.to_csv('../../../results/CNN_conv_1_node_importance_cifar.csv')
NC_2.to_csv('../../../results/CNN_conv_2_node_importance_cifar.csv')
NC_3.to_csv('../../../results/CNN_conv_3_node_importance_cifar.csv')
NC_4.to_csv('../../../results/CNN_conv_4_node_importance_cifar.csv')

In [61]:
NC_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.00000
mean,1.600000,1.200000,29.20000
std,1.646545,0.788811,1.47573
min,0.000000,0.000000,27.00000
25%,0.250000,1.000000,29.00000
50%,1.000000,1.000000,29.00000
75%,2.750000,2.000000,30.50000
max,5.000000,2.000000,31.00000


In [62]:
NC_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,10.000000,3.900000,50.100000
std,2.160247,3.446415,3.842742
min,8.000000,1.000000,43.000000
25%,8.250000,2.000000,47.750000
50%,9.500000,2.500000,51.500000
75%,11.000000,3.750000,52.750000
max,15.000000,12.000000,54.000000


In [63]:
NC_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,27.500000,10.800000,89.700000
std,7.199537,6.941021,8.485936
min,10.000000,3.000000,75.000000
25%,26.000000,5.250000,85.500000
50%,27.500000,8.500000,89.000000
75%,31.750000,17.250000,93.750000
max,35.000000,22.000000,107.000000


In [64]:
NC_4.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.00000,10.000000
mean,75.700000,23.90000,156.400000
std,10.499206,12.72312,19.288454
min,61.000000,9.00000,115.000000
25%,69.250000,14.00000,148.500000
50%,73.000000,21.50000,162.500000
75%,83.750000,30.25000,169.750000
max,92.000000,49.00000,176.000000


In [65]:
ND.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,38.100000,0.800000,25.100000
std,3.665151,0.918937,3.695342
min,33.000000,0.000000,19.000000
25%,35.500000,0.000000,23.000000
50%,38.500000,0.500000,24.500000
75%,40.000000,1.750000,28.250000
max,44.000000,2.000000,31.000000


## Importance based on validation

## Single-Layer ANN

### MNIST

## Set seed

In [42]:
np.random.seed(2020)

In [43]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [44]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [45]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_val, y_val), epochs=5)
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_val, y_val)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [14:05<00:00, 16.90s/it]


In [46]:
NI = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI.to_csv('../../../results/single_layer_node_importance_val_mnist.csv')
NI.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.120000,24.640000,101.240000
std,1.598979,6.467375,6.202567
min,0.000000,11.000000,86.000000
25%,1.000000,19.000000,96.250000
50%,2.000000,24.000000,102.000000
75%,3.000000,29.750000,105.000000
max,5.000000,39.000000,112.000000


### Fashion MNIST

## Set seed

In [47]:
np.random.seed(2020)

In [48]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [49]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [50]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    model, r = models.train_basic_ANN(x_train, y_train, 128, (x_val, y_val), epochs=5)
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_val, y_val)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [14:03<00:00, 16.88s/it]


In [51]:
NI = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI.to_csv('../../../results/single_layer_node_importance_val_fmnist.csv')
NI.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.00000,50.000000,50.000000
mean,15.32000,27.540000,85.140000
std,3.04014,6.788977,6.455531
min,10.00000,14.000000,71.000000
25%,13.00000,22.250000,80.500000
50%,15.00000,26.500000,86.000000
75%,18.00000,32.750000,90.750000
max,21.00000,41.000000,97.000000


## Multi-layer Perceptron

### MNIST

## Set seed

In [52]:
np.random.seed(2020)

In [53]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [54]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [55]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [32, 64, 128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_val, y_val)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [19:16<00:00, 23.14s/it]


In [56]:
NI_1 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_2 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_3 = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_1.to_csv('../../../results/multi_layer_1_node_importance_val_mnist.csv')
NI_2.to_csv('../../../results/multi_layer_2_node_importance_val_mnist.csv')
NI_3.to_csv('../../../results/multi_layer_3_node_importance_val_mnist.csv')

In [57]:
NI_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.00000,50.000000,50.000000
mean,3.14000,33.460000,91.400000
std,1.73805,6.506543,6.243364
min,0.00000,21.000000,78.000000
25%,2.00000,29.000000,86.000000
50%,3.00000,33.000000,92.000000
75%,4.00000,38.750000,96.000000
max,8.00000,48.000000,102.000000


In [58]:
NI_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.220000,16.380000,45.400000
std,1.741334,4.040055,4.199125
min,0.000000,7.000000,35.000000
25%,1.000000,14.000000,42.000000
50%,2.000000,16.000000,46.000000
75%,3.000000,19.000000,48.000000
max,9.000000,27.000000,56.000000


In [59]:
NI_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.400000,4.580000,25.020000
std,1.498298,2.416694,2.316665
min,0.000000,1.000000,19.000000
25%,1.000000,3.000000,23.250000
50%,2.000000,4.500000,25.500000
75%,4.000000,5.000000,27.000000
max,5.000000,11.000000,30.000000


### Fashion MNIST

## Set seed

In [60]:
np.random.seed(2020)

In [61]:
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [62]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [63]:
trials = 50
low_tol = -1e-5
high_tol = 1e-5
layer_sizes = [32, 64, 128]
node_importance = np.zeros((trials, len(layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_val, y_val)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [19:13<00:00, 23.06s/it]


In [64]:
NI_1 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_2 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_3 = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NI_1.to_csv('../../../results/multi_layer_1_node_importance_val_fmnist.csv')
NI_2.to_csv('../../../results/multi_layer_2_node_importance_val_fmnist.csv')
NI_3.to_csv('../../../results/multi_layer_3_node_importance_val_fmnist.csv')

In [65]:
NI_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,17.780000,27.220000,83.000000
std,3.442175,6.270403,6.877945
min,12.000000,7.000000,66.000000
25%,16.000000,24.000000,78.250000
50%,17.000000,27.000000,84.000000
75%,19.000000,31.750000,86.750000
max,27.000000,39.000000,105.000000


In [66]:
NI_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,3.640000,16.920000,43.440000
std,1.881814,4.814265,4.849406
min,0.000000,7.000000,32.000000
25%,3.000000,14.000000,39.000000
50%,3.000000,17.000000,44.000000
75%,4.000000,20.000000,46.000000
max,8.000000,27.000000,55.000000


In [67]:
NI_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,50.000000,50.000000,50.000000
mean,2.800000,5.140000,24.060000
std,1.653691,2.294714,2.444569
min,0.000000,1.000000,18.000000
25%,2.000000,3.000000,22.000000
50%,2.000000,5.000000,24.500000
75%,4.000000,6.750000,26.000000
max,6.000000,10.000000,29.000000


## Convolutional Neural Network

### MNIST

## Set seed

In [68]:
np.random.seed(2020)

In [69]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [70]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [71]:
trials = 10
low_tol = -1e-5
high_tol = 1e-5
conv_layer_sizes = [256, 128, 64, 32]
dense_layer_sizes = [64]
node_importance = np.zeros((trials, len(dense_layer_sizes) + len(conv_layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance_conv(model, tester_model, dense_layer_sizes, conv_layer_sizes, low_tol, high_tol, x_val, y_val)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [13:17<00:00, 79.74s/it]


In [72]:
ND = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_4 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_3 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_2 = pd.DataFrame(node_importance[:, 3, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_1 = pd.DataFrame(node_importance[:, 4, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
ND.to_csv('../../../results/CNN_dense_node_importance_val_mnist.csv')
NC_1.to_csv('../../../results/CNN_conv_1_node_importance_val_mnist.csv')
NC_2.to_csv('../../../results/CNN_conv_2_node_importance_val_mnist.csv')
NC_3.to_csv('../../../results/CNN_conv_3_node_importance_val_mnist.csv')
NC_4.to_csv('../../../results/CNN_conv_4_node_importance_val_mnist.csv')

In [73]:
NC_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,2.500000,14.800000,14.700000
std,1.433721,3.119829,3.772709
min,0.000000,10.000000,9.000000
25%,2.000000,14.000000,12.500000
50%,2.500000,14.500000,14.500000
75%,3.000000,17.500000,15.750000
max,5.000000,19.000000,22.000000


In [74]:
NC_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,1.700000,29.200000,33.100000
std,1.159502,3.614784,4.067486
min,0.000000,23.000000,28.000000
25%,1.000000,26.500000,30.250000
50%,2.000000,30.000000,32.000000
75%,2.750000,32.000000,36.000000
max,3.000000,34.000000,41.000000


In [75]:
NC_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,10.900000,54.500000,62.600000
std,4.581363,8.631338,8.707596
min,2.000000,40.000000,47.000000
25%,9.000000,50.250000,57.250000
50%,11.000000,54.000000,66.000000
75%,13.500000,58.000000,68.750000
max,18.000000,72.000000,71.000000


In [76]:
NC_4.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,55.800000,120.700000,79.500000
std,8.991354,13.832731,16.847354
min,38.000000,103.000000,50.000000
25%,50.500000,111.500000,70.500000
50%,57.500000,117.500000,83.000000
75%,59.500000,128.000000,89.750000
max,68.000000,148.000000,105.000000


In [77]:
ND.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,25.000000,16.700000,22.300000
std,3.972125,4.967673,4.083844
min,20.000000,11.000000,13.000000
25%,22.250000,14.000000,21.000000
50%,24.500000,15.500000,23.000000
75%,28.250000,18.750000,24.000000
max,31.000000,28.000000,28.000000


### Fashion MNIST

## Set seed

In [78]:
np.random.seed(2020)

In [79]:
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [80]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [81]:
trials = 10
low_tol = -1e-5
high_tol = 1e-5
conv_layer_sizes = [256, 128, 64, 32]
dense_layer_sizes = [64]
node_importance = np.zeros((trials, len(dense_layer_sizes) + len(conv_layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance_conv(model, tester_model, dense_layer_sizes, conv_layer_sizes, low_tol, high_tol, x_val, y_val)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [13:19<00:00, 79.99s/it]


In [82]:
ND = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_4 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_3 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_2 = pd.DataFrame(node_importance[:, 3, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_1 = pd.DataFrame(node_importance[:, 4, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
ND.to_csv('../../../results/CNN_dense_node_importance_val_fmnist.csv')
NC_1.to_csv('../../../results/CNN_conv_1_node_importance_val_fmnist.csv')
NC_2.to_csv('../../../results/CNN_conv_2_node_importance_val_fmnist.csv')
NC_3.to_csv('../../../results/CNN_conv_3_node_importance_val_fmnist.csv')
NC_4.to_csv('../../../results/CNN_conv_4_node_importance_val_fmnist.csv')

In [83]:
NC_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.00000,10.000000
mean,0.400000,12.20000,19.400000
std,0.699206,2.65832,2.503331
min,0.000000,8.00000,16.000000
25%,0.000000,11.00000,17.500000
50%,0.000000,11.50000,20.000000
75%,0.750000,14.25000,20.750000
max,2.000000,16.00000,24.000000


In [84]:
NC_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.00000,10.000000
mean,1.400000,25.60000,37.000000
std,1.173788,6.25744,5.830952
min,0.000000,16.00000,29.000000
25%,1.000000,22.25000,34.250000
50%,1.000000,26.00000,35.500000
75%,2.000000,29.00000,39.750000
max,4.000000,34.00000,46.000000


In [85]:
NC_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.00000
mean,10.800000,47.800000,69.40000
std,2.898275,7.800285,6.41526
min,7.000000,36.000000,58.00000
25%,8.250000,43.250000,66.00000
50%,11.500000,45.000000,71.00000
75%,12.750000,53.500000,72.75000
max,15.000000,60.000000,79.00000


In [86]:
NC_4.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.00000,10.00000,10.000000
mean,44.90000,133.60000,77.500000
std,5.15213,21.81335,19.653951
min,36.00000,103.00000,42.000000
25%,41.50000,119.00000,66.500000
50%,45.50000,131.50000,79.500000
75%,49.00000,149.25000,85.750000
max,52.00000,170.00000,106.000000


In [87]:
ND.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,27.100000,12.500000,24.400000
std,4.332051,3.749074,2.458545
min,20.000000,6.000000,20.000000
25%,24.500000,9.750000,23.250000
50%,26.500000,13.000000,24.500000
75%,29.750000,14.000000,25.750000
max,35.000000,19.000000,29.000000


## CIFAR10

## Set seed

In [88]:
np.random.seed(2020)

In [89]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [90]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [91]:
trials = 10
low_tol = -1e-5
high_tol = 1e-5
conv_layer_sizes = [256, 128, 64, 32]
dense_layer_sizes = [64]
node_importance = np.zeros((trials, len(dense_layer_sizes) + len(conv_layer_sizes), 3))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    model.fit(x_train, y_train, epochs=5, verbose=0)
    
    tmp_a, _ = estimate_node_importance_conv(model, tester_model, dense_layer_sizes, conv_layer_sizes, low_tol, high_tol, x_val, y_val)
    for i, (nz, nw, ni) in enumerate(tmp_a):
        node_importance[trial, i, 0] = nz
        node_importance[trial, i, 1] = nw
        node_importance[trial, i, 2] = ni

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [18:08<00:00, 108.89s/it]


In [92]:
ND = pd.DataFrame(node_importance[:, 4, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_4 = pd.DataFrame(node_importance[:, 3, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_3 = pd.DataFrame(node_importance[:, 2, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_2 = pd.DataFrame(node_importance[:, 1, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
NC_1 = pd.DataFrame(node_importance[:, 0, :], columns=['Zero Nodes', 'Worse Nodes', 'Important Nodes'])
ND.to_csv('../../../results/CNN_dense_node_importance_val_cifar.csv')
NC_1.to_csv('../../../results/CNN_conv_1_node_importance_val_cifar.csv')
NC_2.to_csv('../../../results/CNN_conv_2_node_importance_val_cifar.csv')
NC_3.to_csv('../../../results/CNN_conv_3_node_importance_val_cifar.csv')
NC_4.to_csv('../../../results/CNN_conv_4_node_importance_val_cifar.csv')

In [93]:
NC_1.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,41.400000,3.600000,19.000000
std,6.003703,1.264911,5.333333
min,33.000000,2.000000,9.000000
25%,37.250000,3.000000,17.250000
50%,41.000000,3.500000,19.500000
75%,44.000000,4.000000,23.250000
max,52.000000,6.000000,25.000000


In [94]:
NC_2.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,70.900000,91.000000,94.100000
std,8.006248,8.881942,7.218033
min,61.000000,71.000000,82.000000
25%,63.250000,87.000000,90.250000
50%,71.000000,93.500000,93.000000
75%,78.750000,97.250000,98.250000
max,80.000000,99.000000,107.000000


In [95]:
NC_3.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.00000
mean,22.500000,29.900000,75.60000
std,8.746428,6.154492,5.25357
min,14.000000,19.000000,66.00000
25%,16.250000,25.750000,73.25000
50%,21.000000,30.000000,74.50000
75%,25.500000,33.250000,78.50000
max,43.000000,40.000000,85.00000


In [96]:
NC_4.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,10.900000,11.400000,41.700000
std,4.677369,7.411702,7.902883
min,2.000000,1.000000,25.000000
25%,9.000000,7.500000,37.250000
50%,10.500000,11.000000,43.000000
75%,14.250000,13.750000,47.750000
max,18.000000,28.000000,51.000000


In [97]:
ND.describe()

,Zero Nodes,Worse Nodes,Important Nodes
count,10.000000,10.000000,10.000000
mean,2.300000,5.900000,23.800000
std,1.251666,1.449138,2.097618
min,0.000000,4.000000,20.000000
25%,2.000000,5.000000,23.000000
50%,2.000000,6.000000,24.000000
75%,3.000000,6.750000,24.750000
max,4.000000,8.000000,28.000000
